In [1]:
import numpy as np
import pandas as pd
import os
import re
from googletrans import Translator

In [2]:
translator = Translator()
translator.translate('Bonjour',src = 'fr',dest = 'en').text

'Hello'

In [3]:
path_to_dir ='isa-data/xlsx'
list_df_final = []
i = -1
name_sciper= {}
codes = {'AR':'Architecture','CGC':'Chemical Engineering and Biotechnology',
         'GC':'Civil Engineering','EL':'Electrical and Electronics Engineering',
         'GM':'Mechanical Engineering','IN':'Computer Science','IF':'Financial engineering',
         'MA':'Mathematics','MT':'Microengineering',
         'MX':'Materials Science and Engineering',
         'SC': 'Communication Systems','SIE':'Environmental Sciences and Engineering',
         'SV':'Life Sciences and Technologies','MTE':'Management','PH':'Physics'}

def getSciper(key):
    global i,name_sciper
    try:
        return name_sciper[key]
    except KeyError:
        name_sciper[key] = i
        ret = i
        i-=1
        return ret
def clean_type(type_):
    global i
    if(type_['Level'] is None):
        return type_
    temp= type_['Level'].split(',')
    type_['section'] = temp[0]
    level = temp[2] if (temp[0] != 'Management') else temp[3]
    type_['Level']= re.sub(r"\xa0(.)*", r"",level).strip()
    if(type(type_['Nationalité']) == str):
        if(('Suisse' in type_['Nationalité']) or ('Swiss' in  type_['Nationalité'])):
            type_['Nationalité']='Swiss'
        else:
            temp = re.split(r"(,+|;+)", type_['Nationalité'])
            if(len(temp) > 1):
                type_['Nationalité']=temp[0].strip()
    exchange = 'Exchange' in type_['section'] or 'Echange' in type_['section']
    type_['Exchange_student'] = exchange
    if(exchange):
        type_['section']=codes[re.sub(r"Exchange|Echange", r"",type_['section']).strip()]
        
    type_['section']= re.sub(r"-(.)*", r"",type_['section']).strip()
    
    if(type(type_['No Sciper']) != int ):
        type_['No Sciper'] = getSciper(type_['Nom Prénom'])
    if type_['Exchange_student']:
        type_['Type']='Exchange'
    elif type_['section'] == 'CMS':
        type_['Type']='CMS'
    elif type_['section'] == 'Passerelle HES':
        type_['Type']= 'Passerelle HES'
    elif 'Master' in type_['Level']:
        type_['Type']= 'Master'
    elif ('Bachelor' in type_['Level']) or ('training course' in type_['Level']):
        type_['Type']= 'Bachelor'
    
    return type_

def process_df(df,year):
    df.columns = df.loc[1,:].tolist()
    df = df[df['Civilité']!='Civilité'].reset_index(drop=True)
    df['Level'] = None
    types = df[ (df['Civilité']!='Miss') & (df['Civilité']!='Mister')]['Civilité']
    types_index = types.index.tolist()
    types = types.tolist()
    for i in range(len(types_index)):
        index_2 = 0
        if(i != len(types_index)-1):
            index_2= types_index[i+1]
        else:
            index_2 = len(df)-1

        df.loc[types_index[i]:index_2,'Level']= types[i]

    df = df.loc[(df['Civilité']=='Miss')|(df['Civilité']=='Mister')].reset_index(drop = True)
    df['Year']= year
    df['section'] = None
    df['Exchange_student']= None
    df['Type']= None

    df = df.apply(clean_type,axis=1)
    return df


for file in os.listdir(path_to_dir):
    df= pd.read_excel(os.path.join(path_to_dir,file))
    year_df = process_df(df,file.split('.')[0])
    list_df_final.append(year_df)



In [4]:
from IPython.display import display, Image
from functools import reduce
def unique_entries(df):
    if((df['Nom Prénom'] != df['Nom Prénom'].iloc[0]).any()):
        display(df)
    ret = df[~(df['Type'].isnull())]
    if(len(ret)>0):
        ret = ret.head(1)
    else:
        ret= df.head(1)
    return ret
list_df_final = list(map(lambda x : (x.groupby(by='No Sciper')).apply(unique_entries),list_df_final))
reduce(lambda x,y:x+y ,list(map(lambda x:len(x),list_df_final)))




69262

In [5]:
list_df_final = list(map(lambda x : x.reset_index(drop = True),list_df_final))
list_df_final[0]

,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,Email,Nationalité,Level,Year,section,Exchange_student,Type
0,Miss,Koubaji Hend,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,-2,NaN,Tunisian,Bachelor semester 5,2009-2010,Physics,False,Bachelor
1,Miss,Tornay Sandrine,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,-1,NaN,suisse,Bachelor semester 1,2009-2010,Mathematics,False,Bachelor
2,Mister,Vaucher Nicolas,NaN,NaN,NaN,NaN,NaN,Leave of absence,NaN,NaN,106628,NaN,suisse,Bachelor semester 3,2009-2010,Microengineering,False,Bachelor
3,Mister,Colliard Bastien,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,112032,NaN,suisse,Bachelor semester 5,2009-2010,Architecture,False,Bachelor
4,Mister,Faivre Gabriel,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,118298,NaN,suisse,Bachelor semester 5,2009-2010,Microengineering,False,Bachelor
5,Mister,Mülhauser Didier,NaN,NaN,Geotechnics,NaN,NaN,Leave of absence,NaN,NaN,128004,NaN,suisse,Master semester 2,2009-2010,Civil Engineering,False,Master
6,Mister,Graf Gérôme,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,128386,gerome.graf@epfl.ch,suisse,Master semester 2,2009-2010,Financial engineering,False,Master
7,Mister,Viso Rocha Alain Manuel,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,128565,NaN,Spanish,Master semester 3,2009-2010,Communication Systems,False,Master
8,Mister,Taboada Carlos,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,128615,NaN,Spanish,Bachelor semester 5,2009-2010,Microengineering,False,Bachelor
9,Mister,Zerdani Jalil,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,129145,NaN,suisse,Bachelor semester 5,2009-2010,Life Sciences and Technologies,False,Bachelor


In [6]:
final_df = pd.concat(list_df_final,ignore_index=True)
final_df.loc[final_df.section == 'Management of Technology','section'] = 'Management'
final_df.loc[final_df.section == 'Systèmes de communication','section'] = 'Communication Systems'
final_df.replace(u'\xa0',u' ', regex=True, inplace=True)
final_df['Nationalité'].replace(u'suisse',u'Swiss', regex=True, inplace=True)
final_df['Nationalité'].replace(u';',u',', regex=True, inplace=True)
bad_entries = final_df[final_df['No Sciper']< 0]
bad_entries_names = bad_entries['Nom Prénom'].unique()
for name in bad_entries_names:
    sciper = final_df[(final_df['No Sciper']>0) & (final_df['Nom Prénom'] == name)]
    if(len(sciper)>0):
        final_df.loc[(final_df['No Sciper']<0) & (final_df['Nom Prénom'] == name),'No Sciper']= (sciper['No Sciper'].reset_index(drop=True))[0]

    
#final_df.to_csv('processed_isa_data.csv')

In [7]:
final_df

,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,Email,Nationalité,Level,Year,section,Exchange_student,Type
0,Miss,Koubaji Hend,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,-2,NaN,Tunisian,Bachelor semester 5,2009-2010,Physics,False,Bachelor
1,Miss,Tornay Sandrine,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,-1,NaN,Swiss,Bachelor semester 1,2009-2010,Mathematics,False,Bachelor
2,Mister,Vaucher Nicolas,NaN,NaN,NaN,NaN,NaN,Leave of absence,NaN,NaN,106628,NaN,Swiss,Bachelor semester 3,2009-2010,Microengineering,False,Bachelor
3,Mister,Colliard Bastien,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,112032,NaN,Swiss,Bachelor semester 5,2009-2010,Architecture,False,Bachelor
4,Mister,Faivre Gabriel,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,118298,NaN,Swiss,Bachelor semester 5,2009-2010,Microengineering,False,Bachelor
5,Mister,Mülhauser Didier,NaN,NaN,Geotechnics,NaN,NaN,Leave of absence,NaN,NaN,128004,NaN,Swiss,Master semester 2,2009-2010,Civil Engineering,False,Master
6,Mister,Graf Gérôme,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,128386,gerome.graf@epfl.ch,Swiss,Master semester 2,2009-2010,Financial engineering,False,Master
7,Mister,Viso Rocha Alain Manuel,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,128565,NaN,Spanish,Master semester 3,2009-2010,Communication Systems,False,Master
8,Mister,Taboada Carlos,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,128615,NaN,Spanish,Bachelor semester 5,2009-2010,Microengineering,False,Bachelor
9,Mister,Zerdani Jalil,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,129145,NaN,Swiss,Bachelor semester 5,2009-2010,Life Sciences and Technologies,False,Bachelor


In [8]:
mapping_code_nat = dict(list(map(lambda x: (x,translator.translate(x,dest = 'en').text) if type(x) == str else (x,x),final_df['Nationalité'].unique().tolist())))


In [9]:

mapping_code_nat['kosovar'] = 'Kosovar'
mapping_code_nat['centrafricaine']= 'Central African'
mapping_code_nat['Philippine']= 'Filipino'
mapping_code_nat['néo-zélandaise'] = 'New Zealander'
mapping_code_nat['liechtensteinoise']='Liechtensteiner'
mapping_code_nat['Qatar']= 'Qatari'
mapping_code_nat['lettonne']= 'Latvian'
mapping_code_nat['Bangladesh']= 'Bangladeshi'
csv_codes = pd.read_csv('isa-data/Countries-List.csv',encoding='utf-16')
csv_codes = csv_codes.set_index('Demonym_1').to_dict('index')

my_dictionary = {k: v['ISO_Code'] for k, v in csv_codes.items()}
final_df['ISO_code']=None
def f(row):
    global mapping_code_nat,csv_code
    if(row['Nationalité'] is not None) and (type(row['Nationalité'])== str):
        row['Nationalité'] = mapping_code_nat[row['Nationalité']]
        row['ISO_code']= csv_codes[row['Nationalité']]['ISO_Code']
    return row
final_df_copy= final_df.apply(f,axis=1)

In [12]:
csv_codes

{'Andorran': {'Demonym_2': nan,
  'Demonym_3': nan,
  'ISO_Code': 'AD',
  'Name': 'Andorra'},
 'Emirian': {'Demonym_2': 'Emirati',
  'Demonym_3': nan,
  'ISO_Code': 'AE',
  'Name': 'United Arab Emirates'},
 'Afghan': {'Demonym_2': 'Afghani',
  'Demonym_3': nan,
  'ISO_Code': 'AF',
  'Name': 'Afghanistan'},
 'Antiguan': {'Demonym_2': nan,
  'Demonym_3': nan,
  'ISO_Code': 'AG',
  'Name': 'Antigua and Barbuda'},
 'Anguillan': {'Demonym_2': nan,
  'Demonym_3': nan,
  'ISO_Code': 'AI',
  'Name': 'Anguilla'},
 'Albanian': {'Demonym_2': 'Alabanian',
  'Demonym_3': nan,
  'ISO_Code': 'AL',
  'Name': 'Albania'},
 'Armenian': {'Demonym_2': 'Hayastani',
  'Demonym_3': nan,
  'ISO_Code': 'AM',
  'Name': 'Armenia'},
 'Angolan': {'Demonym_2': nan,
  'Demonym_3': nan,
  'ISO_Code': 'AO',
  'Name': 'Angola'},
 'Antarctic': {'Demonym_2': nan,
  'Demonym_3': nan,
  'ISO_Code': 'AQ',
  'Name': 'Antarctica'},
 'Argentinian': {'Demonym_2': 'Argentinian',
  'Demonym_3': 'Argentinean ',
  'ISO_Code': 'AR',


In [9]:
import json
nat_code = {v['ISO_Code']: k for k, v in csv_codes.items()}
js = json.dumps(nat_code)
fp = open('nat_code.json', 'a')
fp.write(js)
fp.close()

nat_code



{'AD': 'Andorran',
 'AE': 'Emirian',
 'AF': 'Afghan',
 'AG': 'Antiguan',
 'AI': 'Anguillan',
 'AL': 'Albanian',
 'AM': 'Armenian',
 'AO': 'Angolan',
 'AQ': 'Antarctic',
 'AR': 'Argentinian',
 'WS': 'Samoan',
 'AT': 'Austrian',
 'AU': 'Australian',
 'AW': 'Arubian',
 'AX': 'Ålandic',
 'AZ': 'Azerbaijani',
 'BA': 'Bosnian',
 'BB': 'Barbadian',
 'BD': 'Bangladeshi',
 'BE': 'Belgian',
 'BF': 'Burkinabe',
 'BG': 'Bulgarian',
 'BH': 'Bahrainian',
 'BI': 'Burundian',
 'BJ': 'Beninese',
 'BL': 'Barthélemois',
 'BM': 'Bermudan',
 'BN': 'Bruneian',
 'BO': 'Bolivian',
 'VA': nan,
 'BR': 'Brazilian',
 'BS': 'Bahameese',
 'BT': 'Bhutanese',
 'BW': 'Motswana',
 'BY': 'Byelorussian',
 'BZ': 'Belizean',
 'CA': 'Canadian',
 'CC': 'Cocossian',
 'CG': 'Congolese',
 'CF': 'Central African',
 'CH': 'Swiss',
 'CI': 'Ivorian',
 'CK': 'Cook Islander',
 'CL': 'Chilean',
 'CM': 'Cameroonian',
 'CN': 'Chinese',
 'CO': 'Colombian',
 'CR': 'Costa Rican',
 'CU': 'Cuban',
 'CV': 'Cape Verdean',
 'CW': 'Curaçaoan',
 

In [69]:
final_df[['No Sciper','Civilité', 'Nom Prénom','Year','Type','section','Nationalité','ISO_code']].to_csv('cleaned_isa_data.csv')

In [15]:
final_df[(final_df['section'] == 'Computer Science') & (final_df['Year'] == '2017-2018' )]


,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,Email,Nationalité,Level,Year,section,Exchange_student,Type,ISO_code
49758,Mister,Salmina Filippo,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,-14,NaN,Swiss,Bachelor semester 1,2017-2018,Computer Science,False,Bachelor,None
49763,Mister,Martin Damien,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,121367,damien.martin@epfl.ch,Swiss,Master semester 1,2017-2018,Computer Science,False,Master,None
49765,Mister,Homberger Armen Cédric,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,154511,armen.homberger@epfl.ch,Swiss,Bachelor semester 1,2017-2018,Computer Science,False,Bachelor,None
49768,Mister,Zimmermann Florian,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,170946,florian.zimmermann@epfl.ch,Swiss,Master semester 3,2017-2018,Computer Science,False,Master,None
49783,Mister,Benhamadi Yassine,NaN,NaN,Internet Information Systems,NaN,NaN,Present,NaN,NaN,185991,yassine.benhamadi@epfl.ch,Tunisian,Master Project autumn,2017-2018,Computer Science,False,Master,None
49785,Mister,Cretegny Benoit Till,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,186790,benoit.cretegny@epfl.ch,Swiss,Master semester 3,2017-2018,Computer Science,False,Master,None
49793,Mister,Bonjour Jonatan,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,189560,jonatan.bonjour@epfl.ch,Swiss,Bachelor semester 1,2017-2018,Computer Science,False,Bachelor,None
49796,Mister,Rubeli Thomas Jean,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,190377,thomas.rubeli@epfl.ch,Swiss,Master semester 3,2017-2018,Computer Science,False,Master,None
49797,Mister,Güzelhan Can,NaN,NaN,NaN,NaN,"Management, Technology and Entrepreneurship minor",Internship,NaN,NaN,192219,can.guezelhan@epfl.ch,Turkish,Master Project autumn,2017-2018,Computer Science,False,Master,None
49799,Mister,Girod Luc,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,192555,luc.girod@epfl.ch,Swiss,Master semester 3,2017-2018,Computer Science,False,Master,None


In [10]:
final_df.section.unique()

array(['Physics', 'Mathematics', 'Microengineering', 'Architecture',
       'Civil Engineering', 'Financial engineering',
       'Communication Systems', 'Life Sciences and Technologies',
       'Computer Science', 'Molecular & Biological Chemistry',
       'Bioengineering', 'Electrical and Electronics Engineering',
       'Chemical Engineering and Biotechnology', 'Mechanical Engineering',
       'Environmental Sciences and Engineering', 'Management',
       'Applied Physics', 'Passerelle HES',
       'Chemistry and Chemical Engineering',
       'Materials Science and Engineering',
       'Micro and Nanotechnologies for Integrated Systems',
       'Nuclear engineering', 'CMS',
       'Computational science and Engineering', 'Applied Mathematics',
       'Mathematics for teaching', 'Energy Management and Sustainability',
       'Chemistry teaching', 'Digital Humanities', 'Data Science'], dtype=object)

In [3]:
list_df_final[1].Level.unique()

array(['Autumn semester', 'Spring semester', 'Master Project autumn',
       'Master Project spring', 'Master semester 1', 'Master semester 2',
       'Master semester 3', 'Master semester 4',
       '3rd year Spring training course',
       '3rd year autumn training course',
       '4th year Automn training course',
       '4th year autumn training course', 'Bachelor semester 1',
       'Bachelor semester 2', 'Bachelor semester 3', 'Bachelor semester 4',
       'Bachelor semester 5', 'Bachelor semester 5b',
       'Bachelor semester 6', 'Bachelor semester 6b', 'Admission autumn',
       'Admission spring'], dtype=object)

In [192]:
final_df['Nationalité'].sort_values().unique()

array(['Afghan', 'Albanian', 'Algerian', 'Algerian, Swiss', 'American',
       'American, Swiss', 'Andorran', 'Angolan', 'Argentinian', 'Armenian',
       'Australian', 'Australian, Swiss', 'Austrian', 'Austrian, Swiss',
       'Azerbaijani', 'Bangladesh', 'Bangladesh, Swiss', 'Belgian',
       'Belgian, Italian', 'Belgian, Swiss', 'Beninese', 'Bolivian',
       'Brazilian', 'Brazilian, Swiss', 'British', 'British, Austrian',
       'British, Dutch', 'British, Israeli', 'British, Italian',
       'British, Swedish', 'British, Swiss', 'Bulgarian', 'Byelorussian',
       'Cambodian, Swiss', 'Cameroonian', 'Cameroonian, Swiss', 'Canadian',
       'Canadian, Swiss', 'Chilean', 'Chinese', 'Chinese, Swiss',
       'Colombian', 'Colombian, Swiss', 'Congolese', 'Costa Rican',
       'Costa Rican, Swiss', 'Croatian', 'Croatian, Swiss', 'Cuban',
       'Cypriot', 'Danish', 'Djiboutian', 'Dutch', 'Dutch, Swiss',
       'Ecuadorian', 'Ecuadorian, Swiss', 'Egyptian', 'Emirian',
       'Estonian', '

In [100]:
match = final_df['Nationalité'].str.match(r'(.)*,(.)*|(.)*;(.)', na=False)
set(final_df[match]['Nationalité'].unique())

set()

In [101]:
final_df['Level'].unique()

array(['Autumn semester', 'Spring semester', 'Master Project autumn',
       'Master semester 1', 'Master semester 2',
       '3rd year Spring training course',
       '3rd year autumn training course',
       '4th year Automn training course',
       '4th year autumn training course', 'Admission autumn',
       'Bachelor semester 1', 'Bachelor semester 2', 'Bachelor semester 3',
       'Bachelor semester 4', 'Bachelor semester 5',
       'Bachelor semester 5b', 'Bachelor semester 6',
       'Bachelor semester 6b', 'Master Project spring',
       'Master semester 3', 'Admission spring', 'Master semester 4',
       'Mise à niveau'], dtype=object)

In [102]:
final_df[final_df.Level =='Communication Systems']

,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,Email,Nationalité,Level,Year,section,Exchange_student


In [8]:
list_ = reduce(lambda x,y :x+y,list(map(lambda x:x['No Sciper'].unique().tolist(),list_df_final ) ))

In [82]:
from itertools import compress
(final_df['Civilité'].reset_index(drop=True))[2]

'Mister'

In [103]:
bad_entries_names = bad_entries['Nom Prénom'].unique()




In [104]:
bad_entries_names

array(['Tornay Sandrine', 'Koubaji Hend', 'Zenkhri Amir',
       'Poirot Charles Thomas', 'Roncal Villanueva Jossef Etsmit',
       'Kerdid Loqmane', 'Soviche Amaury François-Xavier Gérard Marc',
       'Ashari Astani Negar', 'Maret Julien Gaël', 'Bütün Hür Ebuzer',
       'Turner Tomas Erland', 'Guerraoui Nada', 'Pollien Jessy',
       'Salmina Filippo', 'Ang Alexandrea',
       'Zindel Johannes Andrea Lazarus', 'Masur Anne-Kathrin',
       'Mauguin Aurélien'], dtype=object)

In [111]:
final_df[final_df['Nom Prénom'] == 'Moussa Abdeljalil'].sort_values(by = 'Year')

,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,Email,Nationalité,Level,Year,section,Exchange_student
13132,Mister,Moussa Abdeljalil,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,250755,abdeljalil.moussa@epfl.ch,Moroccan,Bachelor semester 1,2014-2015,Communication Systems,False
13219,Mister,Moussa Abdeljalil,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,250755,abdeljalil.moussa@epfl.ch,Moroccan,Bachelor semester 2,2014-2015,Communication Systems,False
80741,Mister,Moussa Abdeljalil,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,250755,abdeljalil.moussa@epfl.ch,Moroccan,Bachelor semester 1,2015-2016,Communication Systems,False
80832,Mister,Moussa Abdeljalil,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,250755,abdeljalil.moussa@epfl.ch,Moroccan,Bachelor semester 2,2015-2016,Communication Systems,False
67005,Mister,Moussa Abdeljalil,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,250755,abdeljalil.moussa@epfl.ch,Moroccan,Bachelor semester 1,2016-2017,Communication Systems,False
67263,Mister,Moussa Abdeljalil,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,250755,abdeljalil.moussa@epfl.ch,Moroccan,Bachelor semester 4,2016-2017,Communication Systems,False
92189,Mister,Moussa Abdeljalil,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,250755,abdeljalil.moussa@epfl.ch,Moroccan,Bachelor semester 3,2017-2018,Communication Systems,False


In [139]:
aggre = final_df.groupby(['Nom Prénom','Level','section']).aggregate('count')
aggre = aggre.loc[(aggre['Civilité']>3)]
aggre[(aggre.index.get_level_values('Level') == 'Bachelor semester 1')]


,,,Civilité,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,Email,Nationalité,Year,Exchange_student
Nom Prénom,Level,section,,,,,,,,,,,,,,
Biro Joseph Erwin,Bachelor semester 1,Mathematics,4,0,0,0,0,0,4,0,0,4,0,4,4,4
Roos Nicola Noah,Bachelor semester 1,Architecture,4,0,0,0,0,0,4,0,0,4,0,4,4,4
Stricker Martin Charles,Bachelor semester 1,Materials Science and Engineering,4,0,0,0,0,0,4,0,0,4,0,4,4,4
Von Zimmermann Anselm,Bachelor semester 1,Architecture,4,0,0,0,0,0,4,0,0,4,4,4,4,4


In [26]:
final_df.loc[(final_df['Type'].isnull()) & (final_df['Level']== 'Mise à niveau')]

,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,Email,Nationalité,Level,Year,section,Exchange_student,Type
54977,Miss,Kamdar Vidhi,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,270446,vidhi.kamdar@epfl.ch,suisse,Mise à niveau,2017-2018,Life Sciences and Technologies,False,None


In [5]:
a = pd.Series([1,2,2,3])

In [6]:
~a.duplicated(keep='first')

0     True
1     True
2    False
3     True
dtype: bool

In [27]:
data = pd.read_csv('processed_isa_data.csv')

/home/ali/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
data[data['No Sciper'] == 270446]

,Unnamed: 0,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,Email,Nationalité,Level,Year,section,Exchange_student,Type
39213,39213,Miss,Kamdar Vidhi,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,270446,vidhi.kamdar@epfl.ch,suisse,Bachelor semester 1,2016-2017,Life Sciences and Technologies,False,Bachelor
54977,54977,Miss,Kamdar Vidhi,NaN,NaN,NaN,NaN,NaN,Present,NaN,NaN,270446,vidhi.kamdar@epfl.ch,suisse,Mise à niveau,2017-2018,Life Sciences and Technologies,False,NaN


In [57]:
pd.Series([None,1,2]).isnull()

0     True
1    False
2    False
dtype: bool